In [193]:
import pandas as pd

# Path to CSV file
spreadspoke_scores_file = "../CSV_Files/spreadspoke_scores.csv"
nfl_teams_file = "../CSV_Files/nfl_teams.csv"
# Read CSV file into DataFrame
spreadspoke_scores_df = pd.read_csv(spreadspoke_scores_file)
nfl_teams_df = pd.read_csv(nfl_teams_file)

# Data inspection
spreadspoke_scores_df

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13796,1/21/2024,2023,Division,True,Buffalo Bills,24,27,Kansas City Chiefs,BUF,-2.5,46,Highmark Stadium,False,25.0,11.0,67.0,NaN
13797,1/21/2024,2023,Division,True,Detroit Lions,31,23,Tampa Bay Buccaneers,DET,-6.0,49.5,Ford Field,False,72.0,0.0,NaN,indoor
13798,1/28/2024,2023,Conference,True,Baltimore Ravens,10,17,Kansas City Chiefs,BAL,-4.5,44,M&T Bank Stadium,False,47.0,7.0,83.0,NaN
13799,1/28/2024,2023,Conference,True,San Francisco 49ers,34,31,Detroit Lions,SF,-7.5,53.5,Levi's Stadium,False,69.0,5.0,55.0,NaN


In [194]:
# Data cleaning and filtering
nfl_teams_df['Abbreviation'] = nfl_teams_df['Abbreviation'].replace('LV', 'LVR')
last_5_years_df = spreadspoke_scores_df[spreadspoke_scores_df['schedule_season'] >= 2019]
clean_df = last_5_years_df.drop(columns=['stadium', 'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
                              'weather_humidity', 'weather_detail', 'spread_favorite', 'over_under_line'])
clean_df


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id
12411,9/5/2019,2019,1,False,Chicago Bears,3,10,Green Bay Packers,CHI
12412,9/8/2019,2019,1,False,Arizona Cardinals,27,27,Detroit Lions,DET
12413,9/8/2019,2019,1,False,Carolina Panthers,27,30,Los Angeles Rams,LAR
12414,9/8/2019,2019,1,False,Cleveland Browns,13,43,Tennessee Titans,CLE
12415,9/8/2019,2019,1,False,Dallas Cowboys,35,17,New York Giants,DAL
...,...,...,...,...,...,...,...,...,...
13796,1/21/2024,2023,Division,True,Buffalo Bills,24,27,Kansas City Chiefs,BUF
13797,1/21/2024,2023,Division,True,Detroit Lions,31,23,Tampa Bay Buccaneers,DET
13798,1/28/2024,2023,Conference,True,Baltimore Ravens,10,17,Kansas City Chiefs,BAL
13799,1/28/2024,2023,Conference,True,San Francisco 49ers,34,31,Detroit Lions,SF


In [195]:
# Count of favorite team appearances
team_favorite_count_df = pd.DataFrame(columns=['Abbreviation', 'Favorite Pick'])
for team_abbreviation in nfl_teams_df['Abbreviation']:
    count = clean_df[clean_df['team_favorite_id'] == team_abbreviation].shape[0]
    team_favorite_count_df = pd.concat([team_favorite_count_df, pd.DataFrame({'Abbreviation': [team_abbreviation], 'Favorite Pick': [count]})])

# Sort teams by favorite count from highest to lowest
team_favorite_count_df = team_favorite_count_df.sort_values(by='Favorite Pick', ascending=False)
team_favorite_count_df

,Abbreviation,Favorite Pick
0,KC,87
0,SF,73
0,BUF,69
0,BAL,66
0,DAL,62
0,TB,60
0,GB,59
0,PHI,58
0,LAR,55
0,NO,54


In [196]:
# Count of favorite team appearances

favorite_wins = clean_df[clean_df["score_home"] > clean_df["score_away"]]["team_favorite_id"].value_counts()
favorite_wins_df = favorite_wins.reset_index()
favorite_wins_df.columns = ['Abbreviation', 'Win Count']
favorite_wins_df

,Abbreviation,Win Count
0,KC,48
1,BUF,41
2,GB,38
3,SF,37
4,DAL,36
5,PHI,34
6,BAL,33
7,CLE,32
8,TB,29
9,LAR,29


In [197]:
# Merge Dateframes
merged_f_t = pd.merge(favorite_wins_df, team_favorite_count_df, on='Abbreviation')
merged_df = pd.merge(merged_f_t, nfl_teams_df, on='Abbreviation')
merged_df = merged_df.reindex(['Name', 'Abbreviation', 'Win Count', 'Favorite Pick', 'Win Rate'], axis=1)

# Calculate win percentage for each team
for row in merged_df:
    ft_win = merged_df['Win Count'].astype(int)
    ft_pick = merged_df['Favorite Pick'].astype(int)
    win_rate = round(ft_win/ft_pick * 100, 2)

# Add Win rate into merge_df dataframe
win_rate_df = merged_df
win_rate_df['Win Rate'] = pd.DataFrame({'Win Rate' : win_rate})
win_rate_df

,Name,Abbreviation,Win Count,Favorite Pick,Win Rate
0,Kansas City Chiefs,KC,48,87,55.17
1,Buffalo Bills,BUF,41,69,59.42
2,Green Bay Packers,GB,38,59,64.41
3,San Francisco 49ers,SF,37,73,50.68
4,Dallas Cowboys,DAL,36,62,58.06
5,Philadelphia Eagles,PHI,34,58,58.62
6,Baltimore Ravens,BAL,33,66,50.00
7,Cleveland Browns,CLE,32,49,65.31
8,Tampa Bay Buccaneers,TB,29,60,48.33
9,Los Angeles Rams,LAR,29,55,52.73


In [198]:
# Calculate Moneyline
def calculate_moneyline(win_rate):
    return 100 / win_rate

# Add Moneyline to Dataframe
win_rate_df['Moneyline'] = win_rate_df['Win Rate'].apply(calculate_moneyline)

# Result
win_rate_df

,Name,Abbreviation,Win Count,Favorite Pick,Win Rate,Moneyline
0,Kansas City Chiefs,KC,48,87,55.17,1.812579
1,Buffalo Bills,BUF,41,69,59.42,1.682935
2,Green Bay Packers,GB,38,59,64.41,1.552554
3,San Francisco 49ers,SF,37,73,50.68,1.973165
4,Dallas Cowboys,DAL,36,62,58.06,1.722356
5,Philadelphia Eagles,PHI,34,58,58.62,1.705902
6,Baltimore Ravens,BAL,33,66,50.00,2.000000
7,Cleveland Browns,CLE,32,49,65.31,1.531159
8,Tampa Bay Buccaneers,TB,29,60,48.33,2.069108
9,Los Angeles Rams,LAR,29,55,52.73,1.896454
